In [1]:
#@title Load Deepjuice and install

from warnings import filterwarnings
from IPython.display import clear_output
import os, sys, getpass
import requests
filterwarnings('ignore')

def is_running_in_colab():
    try: # import test
        import google.colab
        return True
    except ImportError:
        return False


# Install pre-reqs
!pip install ftfy regex tqdm

# Next clone DeepJuiceDev repo
%cd /content/
target_dir = '/content/DeepJuiceDev'
if is_running_in_colab():
    if not os.path.exists(target_dir):
        personal_access_token = getpass.getpass('Enter your Personal Access Token: ')
        git_location = 'github.com/ColinConwell/DeepJuiceDev.git'
        git_url = f"https://{personal_access_token}@{git_location}"
        !git clone $git_url # Execute command to download the repo
        %cd /content/DeepJuiceDev/
        !git checkout latest
    if not os.getcwd() == target_dir:
        os.chdir(target_dir) # change the directory to DeepJuiceDev
    !pip install -q timm mtalg torchmetrics torchinfo
    !pip install -q git+https://github.com/johnmarktaylor91/torchlens

! pip install pytorchvideo


# Finally install deepjuice locally using the setup file
with open('/content/DeepJuiceDev/setup.py', 'w') as f:
  f.write('''
from setuptools import find_packages, setup

setup(
  name='deepjuice',
  packages=find_packages(),
  version='0.1.0',
  description='deepjuice from Colin',
  author='Colin Conwell'
)

''')

%cd /content/DeepJuiceDev
!pip install -e .
%cd /content/
clear_output(wait=True)
print("- Deepjuice repo cloned!")
print("- Deepjuice installed!")
print("!!! RESTART SESSION AND CONTINUE !!!")

- Deepjuice repo cloned!
- Deepjuice installed!
!!! RESTART SESSION AND CONTINUE !!!


In [2]:
#@title Helper functions:
from pathlib import Path
import pandas as pd
import os
import time
import torch
try:
    from deepjuice.procedural.datasets import get_data_loader
except Exception as err:
    print("If No module named 'deepjuice' is the error, make sure to restart session then re-run this cell")
    raise err
from deepjuice.extraction import FeatureExtractor
from deepjuice.systemops.devices import cuda_device_report
from tqdm import tqdm
import numpy as np
try:
    from deepjuice.procedural.datasets import CustomDataset as CustomDataset
except:
    from deepjuice.procedural.datasets import CustomData as CustomDataset
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    NormalizeVideo,
    CenterCropVideo,
)

try:
    from pytorchvideo.transforms import (
        ApplyTransformToKey,
        ShortSideScale,
        UniformTemporalSubsample,
        UniformCropVideo,
    )
except Exception as err:
    print("If No module named 'torchvision.transforms.functional_tensor' is the error: \n Click the link to: \n /usr/local/lib/python3.10/dist-packages/pytorchvideo/transforms/augmentations.py and replace line 9: \n import torchvision.transforms.functional_tensor as F_t \n and change to \n import torchvision.transforms.functional as F_t \n Save and re-run!")
    raise err
from pytorchvideo.data.encoded_video import EncodedVideo
from torch.utils.data import DataLoader


def get_model(model_name):
    """
    Returns the model, transformations and clip duration for the given model name.
    """
    if model_name.lower() in torch.hub.list('facebookresearch/pytorchvideo', force_reload=True):
        model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True).to("cuda").eval()

        # Slowfast specific code
        if model_name == 'slowfast_r50':
            sampling_rate = 2
            fps = 30
            num_frames = 32
            clip_duration = (num_frames * sampling_rate) / fps
            transforms = slowfast_transform(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225], num_frames=num_frames,
                                      side_size=256, crop_size=256)
        else:
            raise Exception(f"{model_name} Transforms are not implemented!")
    else:
        raise Exception(f"{model_name} is not implemented!")
    return model, transforms, clip_duration

class VideoData(CustomDataset):
    def __init__(self, video_paths, clip_duration,
                 transforms=None, device='cuda', **kwargs):
        self.videos = video_paths
        self.clip_duration = clip_duration
        self.device = device
        self.transforms = transforms
        self.data = self.videos

    def __getitem__(self, index):
        video = EncodedVideo.from_path(self.videos[index])  # Initialize an EncodedVideo helper class
        video_data = video.get_clip(start_sec=0, end_sec=self.clip_duration)  # Load the desired clip
        video_data = self.transforms(video_data)  # Transform the video
        inputs = video_data["video"]
        # Move to device
        if isinstance(inputs, torch.Tensor):
            inputs = inputs.to(self.device)
        else:
            inputs = [x.to(self.device) for x in inputs]
        return inputs

    def __len__(self):
        return len(self.videos)

    def get_sample(self, index=None, show_original=False):
        index = self.get_sample_index(index, len(self))

        if show_original:
            print('not yet implemented')

        return self[index]  # the output of __getitem__

def get_video_loader(video_set, clip_duration, transforms, batch_size=64, **kwargs):
  if isinstance(video_set, pd.Series) or isinstance(video_set, list):
    return DataLoader(VideoData(video_set, clip_duration, transforms), batch_size, **kwargs)

####################
# SlowFast transform
####################

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """

    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        alpha = 4
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list


def slowfast_transform(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225], num_frames=32, side_size=256, crop_size=256):
    return ApplyTransformToKey(
        key="video",
        transform=Compose([
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
        )
    )


In [ ]:
#@title Set up variables:

# Model
model_name = 'slowfast_r50'

# Change file paths
data_dir = "/content/drive/MyDrive/Colab_Notebooks/data"
video_path = f'{data_dir}/asl/831/'

# Variables
extension = 'mp4'
device = 'cuda'
cache = '/content/.cache'
memory_limit = None

In [5]:
#@title Extract features:

# Load the data
video_names = [f for f in os.listdir(video_path) if f.endswith(extension)]
stimulus_data = pd.DataFrame(video_names, columns=['video_name'])
stimulus_data['stimulus_path'] = video_path + stimulus_data['video_name']

model, preprocess, clip_duration = get_model(model_name)

dataloader = get_video_loader(stimulus_data['stimulus_path'],
                              clip_duration,
                              preprocess,
                              batch_size=5)

# Feature Extractor
if memory_limit is None:
    # Calculate the memory limit and generate the feature_extractor
    total_memory_string = cuda_device_report(to_pandas=True)[0]['Total Memory']
    total_memory = int(float(total_memory_string.split()[0]))
    memory_limit_int = int(total_memory * 0.75)
    memory_limit = f'{memory_limit_int}GB'

# Custom forward function for deepjuice
def forward_fn(model, x):
    return model(x)

kwargs = {'forward_fn': forward_fn}

try:
    feature_map_extractor = FeatureExtractor(model, dataloader, memory_limit=memory_limit, device=device, initial_report=False, flatten=True, progress=True, **kwargs)
except Exception as err:
    print("\nIf name 'get_compute_device' is not defined is error: \n Click the link to /content/DeepJuiceDev/deepjuice/extraction.py \n Replace line 76: self.output_device = get_compute_device(output_device) \n with: self.output_device = 'cuda' \n\n On line 57 insert: self.memory_limit = memory_limit \n SAVE AND RE-RUN")
    del dataloader
    raise err

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


Extracting sample maps with torchinfo:
  (Moving tensors from CUDA:0 to CPU)


In [6]:
feature_map_extractor

FeatureExtractor Handle for Net
  426 feature maps (+93 duplicates); 100 inputs
  Memory required for full extraction: 289.201 GB
  Memory usage limiting device set to: cuda
  Memory usage limit currently set to: 11.000 GB
  28 batches required for current memory limit 
   Batch-001: 6 feature maps; 8.399 GB 
   Batch-002: 10 feature maps; 10.936 GB 
   Batch-003: 5 feature maps; 8.984 GB 
   Batch-004: 3 feature maps; 9.375 GB 
   Batch-005: 8 feature maps; 10.938 GB 
   Batch-006: 3 feature maps; 9.375 GB 
   Batch-007: 8 feature maps; 10.938 GB 
   Batch-008: 3 feature maps; 9.375 GB 
   Batch-009: 8 feature maps; 10.938 GB 
   Batch-010: 5 feature maps; 10.936 GB 
   Batch-011: 11 feature maps; 10.154 GB 
   Batch-012: 11 feature maps; 10.154 GB 
   Batch-013: 11 feature maps; 10.154 GB 
   Batch-014: 7 feature maps; 10.546 GB 
   Batch-015: 7 feature maps; 10.934 GB 
   Batch-016: 10 feature maps; 10.934 GB 
   Batch-017: 13 feature maps; 10.935 GB 
   Batch-018: 13 feature maps; 

,uid,shape,memory,batch
0,Net,"[100, 400]",160.0 KB,0
1,MultiPathWayWithFuse-2-1-S1,"[100, 80, 8, 64, 64]",1000.00 MB,0
2,MultiPathWayWithFuse-2-1-S2,"[100, 8, 32, 64, 64]",400.00 MB,0
3,ResNetBasicStem-4-1,"[100, 64, 8, 64, 64]",800.00 MB,0
4,Conv3d-5-1,"[100, 64, 8, 128, 128]",3.125 GB,0
...,...,...,...,...
421,AvgPool3d-4-24,"[100, 256, 1, 2, 2]",400.0 KB,27
422,ResNetBasicHead-2-7,"[100, 400]",160.0 KB,27
423,Dropout-3-12,"[100, 2304, 1, 2, 2]",3.52 MB,27
424,Linear-3-13,"[100, 1, 2, 2, 400]",620.0 KB,27
